# Midas Tutorial

Hello! Please follow the tutorial to learn the basics of Midas. Be sure to play around until you are comfortable. You will have about 20 minutes. Should you have any questions, please feel free to ask Yifan, who will be present during the entire session.  You should

* execute the cells
* optionally read the text in <font color="gray">gray</font>.

In [ ]:
from midas import Midas
import numpy as np

m = Midas()

## Initiate Midas

Import the the Midas class from library `midas`. When you create an instance of `Midas`, you see that a dashboard-like area pops up to the right, composed of **two pannels**, one that will be populated with visualizations of dataframes you create, and one for the columns of these dataframes.

**Adjusting the pannels**

- Drag the left edge of the pannel to change the size.
- Click the "toggle midas" and "toggle column shelf" to hide/show the shelves.

<font color='gray'>Per a single notebook, you can only have one Midas instance.</font> 

## Load data

Midas takes in data from a few APIs, such as `from_file`, used below, which loads csv files. These externally loaded dataframees are considered to be **original dataframe**s and will be shown to the yellow pane to the right. <font color="gray"> Any dataframes computed from these original ones will _not_ be shown in the pane, unless you manually add, using `.show_profile`.</font>

In [ ]:
fires_df = m.from_file("./data/fires.csv")
fires_df

## Querying data

Midas supports dataframes with syntax using that of the [data science module](http://data8.org/datascience/), which we talked about earlier.

In [ ]:
# You might be more used to seeing the syntax df['FIRE_SIZE'] > 100
# Our syntax requires that you explicitly call the function.
big_fires = fires_df.where('FIRE_SIZE', m.are.above(100))
big_fires

## Seeing data

Midas attempts to visualize the dataframe for you directly---to do this, just call the `.vis()` function.

If you click on the circular ellipses to the right, you will find a few helpful buttons
- ⬅️➡️ move the charts around in the dashboard area
- 📷 take a snapshot of the current chart
- 📊 navigate back to the cell for which the chart is defined
- ➖ minimizes the chart
- ❌ deletes the chart

<font color="gray">Sometimes, you may want to change the kind of chart, which is also very easy to do in Midas. If you want to change the chart types from bar to scatter, or to line etc., just specify `mark` (`"bar"`, `"line"`, or `"circle"`).</font>

In [ ]:
STATE_distribution = fires_df.group('STATE')
STATE_distribution.vis()

In [ ]:
# you can use your own aggregators
average_fire_size = fires_df.select(['STATE', 'FIRE_SIZE']).group('STATE', np.average)
average_fire_size.vis()

## Seeing distributions automatically

When you click on a column in the columns pane, go ahead and click on the "STATE" column. After you click, two effects take place:
* a chart is created. <font color="gray">Sometimes, we cannot visualize the data if there are too many values or `NaN`s.</font>
* a new cell is created with code that draws the charts, annotated with 🟠. <font color="gray">You can modify these cells---they are the same as any cells you write. If you can a different chart, do modify the queries.</font>

## Making selections

You can **select** within the charts created.

* When the data is numeric, **brush** to select (**shift + drag**).
* When data is string, **click** to select and **shift-click** to add more.

To deselect, clicking on any region of chart that's empty.

When you perform a selection, you will observe two effects
1. a cell will be generated with the selections you have made, annotated with 🔵---if you find these selections too confusing, you can **toggle all selection cells** by click on the 🔵 button in the menu bar.
2. other visualizations will be  filtered by the selection---this is known as a **"cross-filter"** interaction technique.

The original data might be useful to provide a stable point of reference. However, if you do _not_ wish to see the original data in the background (in a dimmer blue color), just click on the emoji 📌 to "unpin" the original data, and then click 📍 to pin the original data back. As an example, when you visualize the `average` or `median`, you might want to remove the orignal data, since the values maybe have increased (as opposed to being smaller).

In [ ]:
m.sel([{"STATE_distribution": {"STATE": ["CA"]}}])

In [ ]:
# reset selections
m.sel([])

### 🌟Interactions Tactics🌟

Filtering data based on your selections could be a powerful tool. Sometimes you have to create the charts so as to filter other data. For instance, if you want to get a sense of how the different `state`s are affected by the cause of fire, you can click on `STATE` and `CAUSE_DESC` to show *both* charts and then click on the causes in the bar chart for `CAUSE_DESC`.

## Accessing selections programmatically

In [ ]:
# the interaction you just had
m.immediate_selection

In [ ]:
# this is a shorthand to just access the value
m.immediate_value

In [ ]:
# across all current charts
m.current_selection

In [ ]:
# all current_selections in the past
m.all_selections

In [ ]:
# if you currently have a filter that applied to STATE_distribution
# you can get the data via the API `get_filtered_data`
STATE_distribution.get_filtered_data()

In [ ]:
# go to the chart dropdown menu, click 📋, which copies the code to the clip-board
STATE_distribution.get_code()

## Reactive cells and custom visualizations

A reactive cell means that Midas will run it after interactions.
Reactive cells can be used to inspect the state or computation related to the selection events.

In [ ]:
%%reactive

print(m.current_selection)

In [ ]:
%%reactive

if m.immediate_value:
    discoverytime_firesize_df = fires_df.where('STATE', m.are.contained_in(m.immediate_value)).select(["DISCOVERY_TIME", "FIRE_SIZE"])
    discoverytime_firesize_df.reactive_vis()